In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
# read in shapefiles with parcel data
downtown = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\years\downtownYEAR.shp")
eedowntown = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\years\eedowntown1YEAR.shp")
nedowntown = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\years\nedowntown1YEAR.shp")
nndowntown = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\years\nndowntown1YEAR.shp")
nwdowntown = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\years\nwdowntown1YEAR.shp")
ssdowntown  = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\years\ssdowntown1YEAR.shp")
sedowntown  = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\years\sedowntown1YEAR.shp")
nedowntown2  = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\years\nedowntown2YEAR.shp")
nndowntown2  = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\years\nndowntown2YEAR.shp")
nwdowntown2  = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\years\nwdowntown2YEAR.shp")

In [3]:
# combine all parcels into a signle geodataframe
combined_gdf = gpd.GeoDataFrame(pd.concat([downtown, eedowntown, nedowntown, nndowntown, nwdowntown,ssdowntown,sedowntown,nedowntown2,nndowntown2,nwdowntown2], ignore_index=True), crs=downtown.crs)


In [4]:
# define a function to calculate the lower end of the decade
def lower_decade(year):
    if year == None:
        return None
    else:
        return (int(year) // 10) * 10

# apply the function to create the 'yrgrouped' column
combined_gdf['yrgrouped'] = combined_gdf['year'].apply(lower_decade)



In [8]:
# read in building footprint data
buildings = gpd.read_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\kingcountybuildings3.shp")

DriverError: R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\kingcountybuildings3.shp: No such file or directory

In [ ]:
# perform spatial join of buildings to parcel data
result = gpd.sjoin(buildings, combined_gdf, how='left', predicate='intersects')

# Define the columns to be joined
join_fields = ['ADDR_FULL','year', 'yrgrouped']

# Select only the necessary columns
result = result[['geometry'] + join_fields]

In [ ]:
# sort the GeoDataFrame based on the 'year' column in descending order
rclean = result.sort_values(by='year', ascending=False)

# drop duplicate geometries, keeping only the first occurrence (most recent year)
rclean = rclean.drop_duplicates(subset='geometry')

# reset index
rclean.reset_index(drop=True, inplace=True)

In [ ]:
# save the geodataframe as a GEOJSON
rclean.to_file(r"R:\GEOG490_590_31955_Spring2024\Student_Data\cflemin2\Final\buildings_clean.geojson")